In [1]:
#Dynamic Pricing for Urban Parking Lots using Pathway + Panel + Bokeh

!pip install pathway panel bokeh --quiet

import pandas as pd
import numpy as np
import pathway as pw
import panel as pn
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import json, os, threading, datetime, warnings

warnings.filterwarnings("ignore")
pn.extension("tabulator")

#Step 1: Load and process dataset
df = pd.read_csv("dataset.csv")
df["datetime"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], dayfirst=True)
df = df.sort_values("datetime")
df["Timestamp"] = df["datetime"].astype(str)

vehicle_map = {"bike": 0.5, "car": 1.0, "truck": 1.5}
traffic_map = {"low": 1, "medium": 2, "high": 3}
df["VehicleWeight"] = df["VehicleType"].map(vehicle_map).fillna(0).astype(float)
df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map).fillna(0).astype(int)

df.to_csv("parking_stream.csv", index=False)

#Step 2: Pathway schema
class ParkingSchema(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str
    VehicleWeight: float
    TrafficLevel: int

#Step 3: Read as static table
table = pw.io.csv.read("parking_stream.csv", schema=ParkingSchema, mode="static")

#Constants
BASE_PRICE = 10.0
ALPHA = 2.0
BETA = 0.5
GAMMA = 0.3
DELTA = 1.5
EPSILON = 1.0
LAMBDA = 0.1
MAX_PRICE = 20.0
MIN_PRICE = 5.0

#Step 4: Pricing function
def compute_price(Occupancy, Capacity, QueueLength, TrafficLevel, IsSpecialDay, VehicleWeight, model_name):
    occ = Occupancy / Capacity
    if model_name == "Linear":
        return min(MAX_PRICE, max(MIN_PRICE, BASE_PRICE + ALPHA * occ))
    elif model_name == "Demand-Based":
        demand = (
            ALPHA * occ + BETA * QueueLength - GAMMA * TrafficLevel +
            DELTA * IsSpecialDay + EPSILON * VehicleWeight
        )
        norm = min(1.0, max(0.0, (demand - 1) / 10))
        return round(min(MAX_PRICE, max(MIN_PRICE, BASE_PRICE * (1 + LAMBDA * norm))), 2)
    elif model_name == "Competitive":
        return round(min(MAX_PRICE, max(MIN_PRICE, BASE_PRICE + np.random.normal(0, 0.5))), 2)
    return BASE_PRICE

#Step 5: Generate pricing JSON for all systems and models
def generate_pricing_json():
    combined = {}
    for sys_id in df["SystemCodeNumber"].unique():
        sys_df = df[df["SystemCodeNumber"] == sys_id].copy()
        records = []
        for i, row in sys_df.iterrows():
            dt = row["datetime"]
            rec = {"datetime": str(dt), "prices": {}}
            for model in ["Linear", "Demand-Based", "Competitive"]:
                price = compute_price(
                    row["Occupancy"], row["Capacity"], row["QueueLength"],
                    row["TrafficLevel"], row["IsSpecialDay"],
                    row["VehicleWeight"], model
                )
                rec["prices"][model] = float(price)
            records.append(rec)
        combined[sys_id] = records

    os.makedirs("outputs", exist_ok=True)
    with open("outputs/pricing_output_all_systems_all_models.json", "w") as f:
        json.dump(combined, f, indent=2)
    print("Combined output generated once at startup: outputs/pricing_output_all_systems_all_models.json")

generate_pricing_json()

#Step 6: Panel widgets
system_selector = pn.widgets.Select(name="Select System Code", options=sorted(df["SystemCodeNumber"].unique().tolist()))
model_selector = pn.widgets.Select(name="Select Pricing Model", options=["Linear", "Demand-Based", "Competitive"])

#Step 7: Bokeh Plot
source = ColumnDataSource(data=dict(datetime=[], price=[]))
plot = figure(title="Dynamic Pricing", x_axis_type="datetime", height=400, width=800)
plot.line('datetime', 'price', source=source, line_width=2)

# Step 8: Update function
def update_plot(event=None):
    system_id = system_selector.value
    model = model_selector.value

    try:
        with open("outputs/pricing_output_all_systems_all_models.json", "r") as f:
            data = json.load(f)

        records = data.get(system_id, [])
        dt = [pd.to_datetime(r["datetime"]) for r in records]
        prices = [r["prices"][model] for r in records]
        source.data = {"datetime": dt, "price": prices}
    except Exception as e:
        print("Error updating plot:", e)

system_selector.param.watch(update_plot, "value")
model_selector.param.watch(update_plot, "value")
update_plot()

# Step 9: Final dashboard layout
dashboard = pn.Column(
    "# Dynamic Pricing Dashboard",
    pn.Row(system_selector, model_selector),
    plot
)

dashboard.servable()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Combined output generated once at startup: outputs/pricing_output_all_systems_all_models.json


Column
    [0] Markdown(str)
    [1] Row
        [0] Select(options=['BHMBCCMKT01', ...], value='BHMBCCMKT01')
        [1] Select(options=['Linear', 'Demand-Based',...], value='Linear')
    [2] Bokeh(figure)